In [1]:
!python --version

Python 3.9.2


In [1]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import datetime
import project_functions2 as pf

from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOGL', 'MSFT', 'TSLA', 'PLTR']#,
              #'NVDA', 'PYPL', 'INTC', 'NFLX', 'ADBE',
              # 'QCOM', 'AMD', 'LULU', 'ZM', 'CRM', 'RIOT',
              #'UBER', 'NIO', 'LYFT', 'SQ', 'SPLK']
#stock_list = ['IDXX', 'PDD', 'ALGN', 'CTSH', 'MTCH', 'ASML',
#              'EA', 'EBAY', 'DOCU', 'MCHP', 'ORLY', 'AVGO'
#              'DXCM', 'CDNS', 'INTU', 'CSCO', , 'SBUX', 'PTON'
#              'ADSK', 'ADI', 'MRNA', 'MAR', 'REGN', 'NXPI',
#              'KHC', 'KDP', 'ROST', 'WBA', 'WDAY', 'BIIB',
#              , 'COST','MU', 'BKNG', 'PEP', 'TMUS', 'TXN'
#              'GILD', 'ISRG', 'MDLZ', 'ADP', 'JD', 'CSX',
#              'ATVI', 'MELI', 'BIDU', 'ILMN', 'VRTX']

#stock_list = []

######## Stocks That arent working ###################
#, 'CMCSA', 'AMGN', 'CHTR' , 'AMAT', 'FISV', 'LRCX', 'MNST'
#, 'KLAC', 'AEP', 'EXC', 'CTAS', 'PAYX', 'NTES', 'ABNB', 'PLTR'
############### Working but Lowers Performance ##############


In [3]:
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

stock_dfs = {}
for key in stock_objects:
    stock_dfs[key] = stock_objects[key].history(period='max')
#googl = yf.Ticker('GOOGL')
#amzn = yf.Ticker('AMZN')
#aapl = yf.Ticker('AAPL')
#fb = yf.Ticker('FB')
#msft = yf.Ticker('MSFT')
#tsla = yf.Ticker('TSLA')
#
#nvda = yf.Ticker('NVDA')
#pypl =  = yf.Ticker('')


In [4]:
#aapl_df = aapl.history(period='max')
#amzn_df = amzn.history(period='max')
#fb_df = fb.history(period='max')
#googl_df = googl.history(period='max')
#msft_df = msft.history(period='max')
#tsla_df = tsla.history(period='max')
for key in stock_dfs:
    stock_dfs[key] = pf.date_time_prep(stock_dfs[key]) 
    stock_dfs[key] = pf.rolling_aves(stock_dfs[key])
    stock_dfs[key] = pf.future_close_setup(stock_dfs[key], 5)

In [5]:
def trading_sim_LRM(combine_df, split_time, stock_dfs, time_shift):
    columns = ['Date', 'Fund Value', 'Cash']
    stock_names = []
    curr_cash = 10000
    curr_shares = {}
    test_dfs = stock_dfs
    
    for key in test_dfs:
        test_dfs[key] = test_dfs[key].tail(split_time)
        curr_shares[key] = 0
        stock_names.append(key)
    
    columns = columns + stock_names
    cash_df = pd.DataFrame(columns=columns)
    curr_line = [combine_df.index[int(len(combine_df) - (split_time * len(stock_dfs)))], curr_cash, curr_cash] + len(stock_names)*[0]
    cash_df.loc[len(cash_df)] = curr_line
    
    while split_time >= time_shift:
        max_stock = ''
        max_stock_gain = 0
        X_train, y_train, X_test, y_test = pf.multi_stock_train_test_split(combine_df, split_time, stock_dfs)
        stock_model = LinearRegression().fit(X_train, y_train)
        
        for key in test_dfs:
            X = test_dfs[key].iloc[:,:-1]
            stock_pred = stock_model.predict(X.head(1))
            stock_pred = (float(stock_pred) - X['Close'][0]) / X['Close'][0]
            if stock_pred > max_stock_gain:
                max_stock = key
                max_stock_gain = stock_pred
        
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[1:]
        
        if max_stock_gain > 0:
            if curr_shares[max_stock] == 0:
                for key in curr_shares:
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0
                curr_shares[max_stock] = curr_cash // test_dfs[max_stock]['Open'][0]
                curr_cash -= curr_shares[max_stock]*test_dfs[max_stock]['Open'][0]
            else:
                curr_shares[max_stock] += curr_cash // test_dfs[max_stock]['Open'][0]
                curr_cash -= (curr_cash // test_dfs[max_stock]['Open'][0]) * test_dfs[max_stock]['Open'][0]
        else:
            for key in curr_shares:
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0
        
        curr_line = [X_test.index[len(stock_names)], curr_cash ,curr_cash] + len(stock_names)*[0]
        cash_df.loc[len(cash_df)] = curr_line
        for key in curr_shares:
            cash_df.iloc[-1, cash_df.columns.get_loc(key)] = curr_shares[key]
            cash_df.iloc[-1, cash_df.columns.get_loc('Fund Value')] += curr_shares[key]*test_dfs[key]['Open'][0]
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[time_shift-1:]
        
        split_time -= time_shift

    return cash_df

In [6]:
stock_dfs = {}
drop_list = ['Stock Splits','Day','Days From IPO','5 Day Volume Mean','5 Day High Var','5 Day Low Var',
             '5 Day Close Var','5 Day Low','10 Day Low Var','10 Day Close Var','10 Day High','10 Day Low',
             '10 Day SMt','20 Day Low Mean','20 Day Low','5 Day Stochastic K','20 Day Stochastic K','5 Day RSI']

for key in stock_objects:
    stock_dfs[key] = stock_objects[key].history(period='max')

for key in stock_dfs:
    stock_dfs[key] = pf.date_time_prep(stock_dfs[key]) 
    stock_dfs[key] = pf.rolling_aves(stock_dfs[key])
    stock_dfs[key] = pf.future_close_setup(stock_dfs[key], 5)
    stock_dfs[key].drop(drop_list, axis=1, inplace=True)


combine_df = pf.combiner(stock_dfs)

cash_df = trading_sim_LRM(combine_df, 510, stock_dfs, 5)
cash_df

ValueError: Found array with 0 sample(s) (shape=(0, 57)) while a minimum of 1 is required.

In [8]:
stock_dfs = {}
drop_list = ['Stock Splits','Day','Days From IPO','5 Day Volume Mean','5 Day High Var','5 Day Low Var',
             '5 Day Close Var','5 Day Low','10 Day Low Var','10 Day Close Var','10 Day High','10 Day Low',
             '10 Day SMt','20 Day Low Mean','20 Day Low','5 Day Stochastic K','20 Day Stochastic K','5 Day RSI']

for key in stock_objects:
    stock_dfs[key] = stock_objects[key].history(period='max')

for key in stock_dfs:
    stock_dfs[key] = pf.date_time_prep(stock_dfs[key]) 
    stock_dfs[key] = pf.rolling_aves(stock_dfs[key])
    stock_dfs[key] = pf.future_close_setup(stock_dfs[key], 20)
    stock_dfs[key].drop(drop_list, axis=1, inplace=True)

combine_df = pf.combiner(stock_dfs)

cash_df = trading_sim_LRM(combine_df, 255, stock_dfs, 20)
cash_df

,Date,Fund Value,Cash,AMZN,AAPL,FB,GOOGL,MSFT,TSLA
0,2020-03-26,10000,10000,0,0,0,0,0,0
1,2020-03-27,10000,1,0,0,0,0,0,99
2,2020-04-27,14605.7,64.7938,0,208,0,0,0,0
3,2020-05-26,16802.6,64.7938,0,208,0,0,0,0
4,2020-06-23,18898,64.7938,0,208,0,0,0,0
5,2020-07-22,20076.2,248.556,0,0,0,0,0,62
6,2020-08-19,23374.6,21.3389,0,202,0,0,0,0
7,2020-09-17,22113.6,1312.25,0,0,0,14,0,0
8,2020-10-15,22935.3,3183.19,6,0,0,0,0,0
9,2020-11-12,22142.9,1238.89,0,0,0,12,0,0
